In [1]:
import numpy as np
seed = 123
np.random.seed(seed)
import collections
from importlib import reload
import pandas as pd
import sklearn
from termcolor import colored # if error: pip3 install termcolor; conda install termcolor 

In [2]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import sklearn.ensemble

In [3]:
import util.data
import util.model
import util.ndcg

In [4]:
# TODO nrows
data_all = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=10*1000)
# data_all = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';')

data_test_unlabelled = pd.read_csv('data/test_set_VU_DM_clean.csv', sep=';', nrows=1000)

In [37]:
for k in data_all.columns:
    if data_all[k].isna().sum() > 0:
#         print('rm %0.4f' % (data_all[k].isna().sum() / data_all.shape[0]), k)
        data_all.drop(columns=[k], inplace=True)

gross_booking_scaler = preprocessing.RobustScaler(copy=False)

# set aside some labelled data for testing (based on srch_id)
ids = data_all.srch_id.unique()
ids_train, ids_test = train_test_split(ids, test_size=0.5, random_state=123)
k = 'gross_bookings_usd'
data_all.loc[ids_train, k] = gross_booking_scaler.fit_transform(data_all.loc[ids_train, [k]])
data_all.loc[ids_test, k] = gross_booking_scaler.transform(data_all.loc[ids_test, [k]])
# add noise to reduce dependency on this attr
data_all.loc[ids_train, k] += np.random.normal(loc=0, scale=0.1, size=ids_train.size)

data = data_all[data_all.srch_id.isin(ids_train)]
data_test = data_all[data_all.srch_id.isin(ids_test)]

# split cross validation folds
folds = util.data.cv_folds_for_sklearn(data, n_cv_folds=5, resampling_ratio=0)

# Use the full (pre-splitted) dataset because we use custom indices
# x_train, y_train will depend on the folds
x, y = util.data.split_xy(data_all)

x_test, y_test = util.data.split_xy(data_test)

In [41]:
kwargs = {'random_state': 1234}
models = {}
params = {}
models['Ensemble Random Forest'] = sklearn.ensemble.RandomForestClassifier(n_jobs=2)
params['Ensemble Random Forest'] = {'n_estimators': [100]}
models['GBoost reg'] = xgb.XGBRegressor(criterion='friedman_mse', loss='ls', tol=1e-4, n_jobs=2, **kwargs)
models['GBoost ndcg'] = xgb.XGBRegressor(objective='rank:ndcg', criterion='friedman_mse', 
                                         loss='ls', tol=1e-4, n_jobs=2, **kwargs)
params['GBoost reg'] = {'n_estimators': [100, 500],
                    'learning_rate': [0.1, 0.01, 0.001],
                    'max_depth': [1,2,3]
                    'base_score': [0, None]
                   }
params['GBoost ndcg'] = params['GBoost reg']
models['AdaBoost'] = sklearn.ensemble.AdaBoostRegressor()
params['AdaBoost'] = {'learning_rate':[1, 0.1],
                     'n_estimators': [100, 500],
                      'loss': ['linear', 'square']
                     }

scores = ['neg_mean_squared_error'] # TODO add ndcg here
for k, model in models.items():
    for score in scores:
        print("\n# %s\t (score: %s)" % (k, score))
        assert k in params.keys(), 'models and params should have the same keys'
        clf = sklearn.model_selection.GridSearchCV(model, params[k], cv=folds, scoring=score, n_jobs=2)
        clf.fit(x, y) # cv/folds is used to select folds
        print("\tBest params (train)", clf.best_params_)
        y_true, y_pred = y_test, clf.predict(x_test)
        mse =  sklearn.metrics.mean_squared_error(y_true, y_pred)
        print(colored('\tmse: %0.4f' % mse, 'green'))
        ndcg = util.ndcg.ndcg(x_test, y_test, y_pred)
        print(colored('\tndcg: %0.4f' % ndcg, 'blue'))

print('done')


# GBoost reg (score: neg_mean_squared_error)
Best params (train)
	 {'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 1000}
mse: 0.3306
ndcg: 0.4022

# GBoost ndcg (score: neg_mean_squared_error)
Best params (train)
	 {'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 1000}
mse: 0.3306
ndcg: 0.4022
done


# Use a subset of features

In [61]:
# TODO nrows
data_all = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=10*1000)
# data_all = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';')

data_test_unlabelled = pd.read_csv('data/test_set_VU_DM_clean.csv', sep=';', nrows=1000)

In [62]:
features = [k for k in data_all.columns if 
            'score' in k or 
            'srch_id' in k or 
            'prop_id' in k or 
            'booking_bool' in k or
            'click_bool' in k or
            'unavailable_comp' in k or
            'available_comp' in k or
            'visitor_hist_starrating' in k or
            'delta_starrating' in k or
            'visitor_hist_adr_usd_log' in k or
            'price_usd_log' in k or
            'month' in k or
            'starrating' in k or
            'promotion' in k or
            'srch_length' in k or
            'distance' in k or
            'gross_booking' in k
           ]
features

['srch_id',
 'visitor_hist_starrating',
 'prop_id',
 'prop_review_score',
 'prop_location_score1',
 'prop_location_score2',
 'promotion_flag',
 'srch_length_of_stay',
 'srch_query_affinity_score',
 'orig_destination_distance',
 'click_bool',
 'gross_bookings_usd',
 'booking_bool',
 'srch_person_per_room_score',
 'srch_adults_per_room_score',
 'delta_starrating',
 'visitor_hist_adr_usd_log',
 'price_usd_log',
 'unavailable_comp',
 'available_comp',
 'month',
 'score',
 'prop_starrating_label0',
 'prop_starrating_label1',
 'prop_starrating_label2',
 'prop_starrating_label3',
 'srch_length_of_stay_bin0',
 'srch_length_of_stay_bin1',
 'srch_length_of_stay_bin2',
 'orig_destination_distance_bin0',
 'orig_destination_distance_bin1',
 'orig_destination_distance_bin2']

In [63]:
for k in data_all.columns:
    if k not in features:
        data_all.drop(columns=[k], inplace=True)
for k in data_test_unlabelled.columns:
    if k not in features:
        data_test_unlabelled.drop(columns=[k], inplace=True)

# data_all = data_all[[k for k in data_all.columns if k in features]]
# data_test_unlabelled = data_test_unlabelled[[k for k in data_test_unlabelled.columns if k in features]]

for k in data_all.columns:
    if data_all[k].isna().sum() > 0:
#         print('rm %0.4f' % (data_all[k].isna().sum() / data_all.shape[0]), k)
        data_all.drop(columns=[k], inplace=True)

gross_booking_scaler = preprocessing.RobustScaler(copy=False)

# set aside some labelled data for testing (based on srch_id)
ids = data_all.srch_id.unique()
ids_train, ids_test = train_test_split(ids, test_size=0.5, random_state=123)
k = 'gross_bookings_usd'
data_all.loc[ids_train, k] = gross_booking_scaler.fit_transform(data_all.loc[ids_train, [k]])
data_all.loc[ids_test, k] = gross_booking_scaler.transform(data_all.loc[ids_test, [k]])
# add noise to reduce dependency on this attr
data_all.loc[ids_train, k] += np.random.normal(loc=0, scale=0.1, size=ids_train.size)

data = data_all[data_all.srch_id.isin(ids_train)]
data_test = data_all[data_all.srch_id.isin(ids_test)]

# split cross validation folds
folds = util.data.cv_folds_for_sklearn(data, n_cv_folds=5, resampling_ratio=0)

# Use the full (pre-splitted) dataset because we use custom indices
# x_train, y_train will depend on the folds
x, y = util.data.split_xy(data_all)

x_test, y_test = util.data.split_xy(data_test)

prop_brand_bool
prop_log_historical_price
srch_booking_window
srch_adults_count
srch_children_count
srch_room_count
srch_saturday_night_bool
random_bool
has_purch_hist_bool
has_historical_price
avg_price_comp
year
day
hour
minute
weekday_Monday
weekday_Other
weekday_Thursday
weekday_Tuesday
weekday_Wednesday
site_id_label0
site_id_label1
site_id_label2
site_id_label3
visitor_location_country_id_label0
visitor_location_country_id_label1
visitor_location_country_id_label2
visitor_location_country_id_label3
prop_country_id_label0
prop_country_id_label1
prop_country_id_label2
prop_country_id_label3
srch_destination_id_label0
srch_destination_id_label1
srch_destination_id_label2
srch_destination_id_label3
srch_adults_count_bin0
srch_adults_count_bin1
srch_adults_count_bin2
srch_children_count_bin0
srch_children_count_bin1
srch_children_count_bin2
srch_room_count_bin0
srch_room_count_bin1
srch_room_count_bin2
srch_booking_window_bin0
srch_booking_window_bin1
srch_booking_window_bin2
visitor_

In [ ]:
kwargs = {'random_state': 1234}
models = {}
params = {}
models['Ensemble Random Forest'] = sklearn.ensemble.RandomForestClassifier(n_jobs=2)
params['Ensemble Random Forest'] = {'n_estimators': [100]}
models['GBoost reg'] = xgb.XGBRegressor(criterion='friedman_mse', loss='ls', tol=1e-4, n_jobs=2, **kwargs)
models['GBoost ndcg'] = xgb.XGBRegressor(objective='rank:ndcg', criterion='friedman_mse', 
                                         loss='ls', tol=1e-4, n_jobs=2, **kwargs)
params['GBoost reg'] = {'n_estimators': [100, 500],
                    'learning_rate': [0.1, 0.01, 0.001],
                    'max_depth': [1,2,3]
                    'base_score': [0, None]
                   }
params['GBoost ndcg'] = params['GBoost reg']
models['AdaBoost'] = sklearn.ensemble.AdaBoostRegressor()
params['AdaBoost'] = {'learning_rate':[1, 0.1],
                     'n_estimators': [100, 500],
                      'loss': ['linear', 'square']
                     }

scores = ['neg_mean_squared_error'] # TODO add ndcg here
for k, model in models.items():
    for score in scores:
        print("\n# %s (score: %s)" % (k, score))
        assert k in params.keys(), 'models and params should have the same keys'
        clf = sklearn.model_selection.GridSearchCV(model, params[k], cv=folds, scoring=score, n_jobs=2)
        clf.fit(x, y) # cv/folds is used to select folds
        print("Best params (train)")
        print('\t', clf.best_params_)
        y_true, y_pred = y_test, clf.predict(x_test)
        mse =  sklearn.metrics.mean_squared_error(y_true, y_pred)
        print(colored('mse: %0.4f' % mse, 'green'))
        ndcg = util.ndcg.ndcg(x_test, y_test, y_pred)
        print(colored('ndcg: %0.4f' % ndcg, 'blue'))

print('done')